# Brand Sentiment Analysis of Twitter Posts

## Setup

In [ ]:
# PULL FROM GIT LARGE FILE STORAGE (LFS)
# !git lfs pull --include="datasets/*"
!git lfs pull --include="cache/*"
!git lfs pull --include="*.mp4"
!git lfs pull --include="Final-Project-Team-2.mp4"

### Package Management

We define a list of required packages and a function for installing them. This function, `handle_packages`, checks for each package in the system. If a package is not found, it is installed quietly using the pip package manager. 

By invoking `handle_packages` with the list of required packages, we make sure all project dependencies are satisfied, setting up a reliable and reproducible environment for our project.

In [ ]:
import os

# INSTALL REQUIRED PACKAGES FOR PROJECT
PACKAGES = [
    "adjustText",
    "colorama",
    "dotenv",
    "ipython",
    "joblib",
    "matplotlib",
    "nltk",
    "numpy",
    "openai",
    "pandas",
    "preprocessor",
    "python-dotenv",
    "seaborn",
    "sklearn",
    "tqdm",
    "tweet-preprocessor",
    "wandb",
    "wordcloud",
]


def handle_packages(packages: list) -> None:
    """Quietly installs a package if it is not already installed.

    Args:
        packages (list): The list of packages to install.

    Raises:
        ImportError: If the package is not installed, install it.
    """
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            os.system(f"python3 -m pip install -q {package}")


handle_packages(PACKAGES)

In [ ]:
import datetime
import itertools
import json
import logging
import pickle
import re
import string
import time
import warnings
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
from enum import Enum
from functools import wraps
from pprint import pprint
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import dotenv
import joblib
import matplotlib.pyplot as plt
import nltk
import numpy as np
import openai
import pandas as pd
import preprocessor
import seaborn as sns
from adjustText import adjust_text
from colorama import Fore, Style
from IPython.display import Markdown, clear_output, display, display_html
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from tqdm import tqdm
from wordcloud import WordCloud

import wandb

warnings.filterwarnings("ignore")


In [ ]:
# SAVE PROJECT VISUALIZATIONS/ARTIFACTS IN THE REPORT DIRECTORY
os.makedirs("report", exist_ok=True)

In [ ]:
class Metrics:
    """
    A class to hold metrics for monitoring performance.

    This class is a singleton, meaning that there is only one instance of the class throughout the entire program.
    All functions decorated with @measure_time share the same Metrics object.

    Attributes:
        metrics (dict): A dictionary to store the metrics.
    """

    _instance = None

    def __new__(cls):
        """
        Overrides the default object creation method to implement the singleton pattern.

        Returns:
            Metrics: The singleton instance of the Metrics class.
        """
        if cls._instance is None:
            cls._instance = super(Metrics, cls).__new__(cls)
            cls._instance.metrics = {}
        return cls._instance

    def __init__(self):
        """Initializes the Metrics class with an empty dictionary."""
        self.metrics: dict[str, float] = {}

    def add_metric(self, key: str, value: float) -> None:
        """
        Adds a new metric to the dictionary or appends to an existing one.

        Args:
            key (str): The name of the metric.
            value (float): The value of the metric.
        """
        if key in self.metrics:
            self.metrics[key] += value
        else:
            self.metrics[key] = value

    def get_metrics(self) -> dict[str, float]:
        """
        Returns the dictionary of metrics.

        Returns:
            dict: The dictionary of metrics.
        """
        return self.metrics

    def __str__(self) -> str:
        """Returns a string representation of the metrics."""
        return "\n".join(f"{k}: {v}" for k, v in self.metrics.items())

    def __getitem__(self, key: str) -> float:
        """
        Returns the value of the metric with the given key.

        Args:
            key (str): The key of the metric.

        Returns:
            float: The value of the metric.
        """
        return self.metrics.get(key)

    def __setitem__(self, key: str, value: float) -> None:
        """
        Sets the value of the metric with the given key.

        Args:
            key (str): The key of the metric.
            value (float): The value of the metric.
        """
        self.metrics[key] = value

    def __len__(self) -> int:
        """
        Returns the number of metrics.

        Returns:
            int: The number of metrics.
        """
        return len(self.metrics)

    def __contains__(self, key: str) -> bool:
        """
        Checks if the metric with the given key exists.

        Args:
            key (str): The key of the metric.

        Returns:
            bool: True if the metric exists, False otherwise.
        """
        return key in self.metrics

    def clear_metrics(self) -> None:
        """Clears all the metrics."""
        self.metrics.clear()

    def remove_metric(self, key: str) -> None:
        """
        Removes the metric with the given key.

        Args:
            key (str): The key of the metric.
        """
        if key in self.metrics:
            del self.metrics[key]

    def update_metric(self, key: str, value: float) -> None:
        """
        Updates the value of the metric with the given key.

        Args:
            key (str): The key of the metric.
            value (float): The new value of the metric.
        """
        if key in self.metrics:
            self.metrics[key] = value

    def print_metrics(self) -> None:
        """Prints all the metrics in a nice format."""
        print("Metrics:")
        sorted_metrics = sorted(
            self.metrics.items(), key=lambda item: item[1], reverse=True
        )
        for key, value in sorted_metrics:
            print(f"  {key}: {value:.2f} seconds")
        print(f"Total time: {sum(self.metrics.values()):.2f} seconds")


def measure_time(run_name: str):
    """
    A decorator factory that measures the time taken by a function and logs it to the Metrics singleton.

    Args:
        run_name (str): The name of the run. This is used to differentiate metrics among different runs.

    Returns:
        Callable: The decorator that wraps the function with timing and logging.
    """

    def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
        """
        The actual decorator that wraps the function with timing and logging.

        Args:
            func (Callable): The function to be timed and logged.

        Returns:
            Callable: The wrapped function.
        """

        def wrapper(*args, **kwargs):
            """
            The wrapper function that adds timing and logging to the original function.

            Args:
                *args: The positional arguments for the original function.
                **kwargs: The keyword arguments for the original function.

            Returns:
                The result of the original function.
            """
            # START TIMER
            start_time = time.time()
            # CALL FUNCTION
            result = func(*args, **kwargs)
            # CALCULATE ELAPSED TIME
            total_time = time.time() - start_time
            # LOG RESULT
            metrics.add_metric(f"{run_name}", total_time)
            print(f"{run_name}...{total_time:.2f} seconds")
            return result

        return wrapper

    return decorator

metrics = Metrics()

### Setting Up Data Analysis and Visualization Tools

We import data handling and visualization libraries and configure them for ideal display. These steps prepare our environment for efficient data analysis and visualization tasks.

In [ ]:
# CONFIGURE
pd.set_option("display.max_colwidth", 80)

# SET MATPLOTLIB STYLE TO 'fivethirtyeight'
plt.style.use("fivethirtyeight")

### Loading Environment Variables

We load environment variables from a `.env` file using the `dotenv` package. This file has key-value pairs of environment variables, which are then added to the system's environment variables. This approach lets us manage sensitive information securely and consistently across different project stages.

In [ ]:
# LOAD ENVIRONMENT VARIABLES
ENVIRONMENT = dotenv.dotenv_values()
for key in ENVIRONMENT:
    os.environ[key] = ENVIRONMENT[key]

## Exploratory Data Analysis (EDA)

### Retrieving and Preparing the Sentiment140 Dataset

The Sentiment140 dataset, available at <http://help.sentiment140.com/for-students>, is a collection of 1.6 million tweets with sentiment labels. 

It is used for discovering the sentiment of a brand, product, or topic on Twitter.

The dataset, provided as a CSV file with emoticons removed, includes tweet content and sentiment labels. We extract these parts and remap the sentiment labels for easier interpretation in future analyses.

The dataset has 6 fields:
1. `target` — the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
1. `id` — the id of the tweet (2087)
1. `date` — the date of the tweet (Sat May 16 23:58:44 UTC 2009)
1. `flag` — the query (lyx). If there is no query, then this value is NO_QUERY.
1. `user` — the user that tweeted (robotickilldozr)
1. `text` — the text of the tweet (Lyx is cool)

In [ ]:
tweets_df = pd.read_csv(
    "datasets/training.1600000.processed.noemoticon.csv",
    encoding="ISO-8859-1",
    names=["sentiment", "ids", "date", "flag", "user", "tweet"],
    header=None,
)

# GET TWEETS AND SENTIMENT LABELS
tweets_df = tweets_df[["tweet", "sentiment"]]

# REMAP SENTIMENT LABELS TO 0 = negative, 1 = positive (instead of 0 = negative, 4 = positive)
tweets_df.sentiment = tweets_df.sentiment.replace(4, 1)

### Class Balance

We ensure class balance in our dataset by counting the number of positive and negative tweets. Class balance is important because the model is trained on an equal number of instances from each class, preventing bias towards a particular sentiment. 

We generate a bar plot showing the distribution of sentiments in the training dataset, which indicates whether models will be trained on a balanced set of samples.

In [ ]:
# SHOW SENTIMENT DISTRIBUTION IN TRAINING SET
distribution = tweets_df.sentiment.value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=distribution.index, y=distribution.values)
plt.title("Distribution of Sentiment in Sentiment140 Training Set")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.xticks([0, 1], ["Negative", "Positive"])

for i, v in enumerate(distribution.values):
    # ADD COUNTS ABOVE BARS
    plt.text(i, v, str(v), ha="center")

# SAVE VISUALIZATION
plt.tight_layout()
plt.savefig("report/sentiment_distribution.png", dpi=300)

plt.show()

### Word Cloud

A word cloud is a visual representation where the most often occurring words are displayed larger than less common words. This visualization gives a quick understanding of the key themes or topics in the text data.

In [ ]:
@measure_time("generating word cloud from training data")
def generate_wordcloud(df_train):
    """
    Generate a wordcloud from a dataframe.

    Args:
        df_train (DataFrame): The dataframe containing the text data.

    Returns:
        None
    """
    wordcloud = WordCloud(
        width=1600,
        height=800,
        background_color="white",
        min_font_size=10,
        max_words=1000,
        collocations=False,
        random_state=42,  # set for idempotency
    )

    wordcloud.generate(" ".join(df_train.tweet.tolist()))

    plt.figure(figsize=(8, 8))
    plt.imshow(wordcloud)
    plt.axis("off")

    # SAVE VISUALIZATION
    plt.tight_layout()
    plt.savefig("report/wordcloud.png", dpi=300)

    plt.show()


generate_wordcloud(tweets_df)

## Preprocessing

Preprocessing is a key step in Natural Language Processing (NLP) to improve the performance and accuracy of the models. The aim is to get the text data ready for machine learning algorithms. 

In its raw form, text data can be messy and unstructured, making it hard for a machine-learning model to understand and learn from. Preprocessing cleans up the data by removing unnecessary information like punctuation and common words that add little meaning or converting all text to lowercase so the model does not get confused by the same word in different cases. Transforming text into a clean and standardized form makes it easier for machine learning models to analyze the data and perform tasks like classification, prediction, or translation.

In [ ]:
BRANDS = [
    "facebook",
    "google",
    "apple",
    "starbucks",
    "disney",
    "microsoft",
    "target",
    "amazon",
    "walmart",
    "sony",
]

In [ ]:
# CALCULATE FREQUENCY OF EACH BRAND IN THE TWEETS
counter = Counter(
    word
    for tweet in tweets_df.tweet
    for word in tweet.lower().split()
    if word in BRANDS
)

# CREATE DATAFRAME FROM THE COUNTER
brand_freqs_df = pd.DataFrame(
    counter.items(), columns=["brand", "frequency"]
).sort_values("frequency", ascending=False)

In [ ]:
# VISUALIZE BRAND MENTION FREQUENCY

# CREATE BARPLOT
num_brands = len(brand_freqs_df)
plt.figure(figsize=(10, num_brands * 0.5))
barplot = sns.barplot(
    x="frequency", y="brand", data=brand_freqs_df, order=brand_freqs_df.brand
)
plt.xticks(rotation=90)
plt.title("Brand Mention Frequency")

# ADD COUNTS AT THE END OF THE BARS
for i, v in enumerate(brand_freqs_df.frequency):
    barplot.text(v + 3, i + 0.25, str(v))

# SAVE VISUALIZATION
plt.tight_layout()
barplot.figure.savefig("report/brand_mention_frequency.png", dpi=300)

plt.show()

### Process Tweets to Identify Brand Mentions

We analyze tweets and identify mentions of specific brands. The process begins by standardizing the text in each tweet, which aids in detecting brand names. Each tweet is analyzed for brand mentions. When a brand is detected, we store the original tweet text, a flag indicating a brand mention, and the brand name itself. Where no brand is mentioned, we record the brand name as `nobrand`.

In [ ]:
@measure_time("creating brands dataframe")
def create_brand_dataframe(tweets_df: pd.DataFrame, brands: List[str]) -> pd.DataFrame:
    """
    Create a new dataframe with brand labels.

    Args:
        tweets_df (pd.DataFrame): The dataframe containing tweets and sentiments.
        brands (List[str]): The list of brands to be searched in the tweets.

    Returns:
        pd.DataFrame: The new dataframe with brand labels.
    """
    brand_rows = []
    non_brand_counter = 0
    brand_counter = 0
    for tweet, sentiment in zip(tweets_df.tweet, tweets_df.sentiment):
        tweet_tokens = tweet.lower().split()
        brand_found = False

        for word in tweet_tokens:
            word = word.replace("-", "")
            for brand in BRANDS:
                if word == brand:
                    brand_rows.append(
                        {"tweet": tweet, "brand": brand, "sentiment": sentiment}
                    )
                    brand_found = True
                    brand_counter += 1
                    break

            if brand_found:
                break

        if not brand_found and non_brand_counter < brand_counter:
            brand_rows.append(
                {"tweet": tweet, "brand": "nobrand", "sentiment": sentiment}
            )
            non_brand_counter += 1

    brands_df = pd.DataFrame(brand_rows, columns=["tweet", "brand", "sentiment"])
    return brands_df


brands_df = create_brand_dataframe(tweets_df, BRANDS)
display(brands_df.head())

In [ ]:
# OUTPUT DATASET TO FILE
print(len(brands_df))
brands_df.to_csv("datasets/brands.csv", index=False)

### Data Generation Using OpenAI's GPT-4 Model

We define parameters for data generation using OpenAI's GPT-4 model. The parameters include the model name, the desired dataset size, and a user prompt that outlines the required data format.

The data format is: 

"tweet"|||brand-presence|||brand

Here, 'tweet' represents a Twitter post, 'brand-presence' is a binary indicator (0 or 1) denoting the presence or absence of a brand in the tweet, and 'brand' is the name of a brand from a predefined list or 'nobrand' when no brand is mentioned in the tweet.

The user prompt also includes examples to guide the data generation process and specifies that the dataset should have an equal number of tweets with and without brand mentions.

We use OpenAI's API to generate the data. The API key is retrieved from the environment variables, and a chat completion task is created using the GPT-4 model and the defined user prompt. The generated data is then extracted from the chat completion response.

In [63]:
logging.basicConfig(filename='data_generator.log', level=logging.INFO)

class Status(Enum):
    NOT_STARTED = "NOT_STARTED"
    QUEUED = "QUEUED"
    BALANCING = "BALANCING"
    BALANCED = "BALANCED"

class JobAction(Enum):
    GENERATE = "GENERATE"
    DELETE = "DELETE"

class Job:
    def __init__(self, brand, sentiment, action: JobAction, count: int):
        self.brand = brand
        self.sentiment = sentiment
        self.action = action
        self.count = count

class DataGenerator:
    def __init__(self, brands, model="gpt-4", samples_per_brand=100, cache_file="cache/generated_data.pkl"):
        self.brands = brands
        self.model = model
        self.samples_per_brand = samples_per_brand
        self.dataset_size = samples_per_brand * (len(brands) * 2 + 1)
        self.dataset = []
        self.seen_data = set()
        self.brand_counts = {brand: {"positive": 0, "negative": 0} for brand in brands}
        self.brand_counts["nobrand"] = {"positive": 0, "negative": 0}
        self.brand_status = {brand: Status.NOT_STARTED for brand in brands}
        self.brand_status["nobrand"] = Status.NOT_STARTED
        openai.api_key = os.environ["OPENAI_API_KEY"]
        self.cache_file = cache_file
        self.load_or_initialize_dataset()
        self.delete_excess_data()
        self.job_queue = Queue()

    def add_job(self, brand, sentiment, action: JobAction):
        count = self.calculate_job_count(brand, sentiment, action.value.lower())
        job = Job(brand, sentiment, action, count)
        self.job_queue.put(job)
        logging.info(f'Job added: {job}')

    def calculate_jobs(self):
        for brand in self.brands + ["nobrand"]:
            for sentiment in ["positive", "negative"]:
                current_count = self.brand_counts[brand][sentiment]
                target_count = self.samples_per_brand // 2
                if current_count < target_count and self.brand_status[brand] != Status.BALANCED:
                    self.add_job(brand, sentiment, JobAction.GENERATE)
                elif current_count > target_count and self.brand_status[brand] != Status.BALANCED:
                    self.add_job(brand, sentiment, JobAction.DELETE)
        self.report_metrics()

    def process_jobs(self):
        while not self.job_queue.empty():
            job = self.job_queue.get()
            self.process_job(job)
            self.job_queue.task_done()
            if self.brand_status[job.brand] == Status.BALANCED: 
                self.job_queue.queue.remove(job)

    def process_job(self, job):
        self.update_status(job.brand, Status.BALANCING)
        if job.action == JobAction.GENERATE:
            if job.sentiment == "positive":
                self.generate_positive_data(job.brand, job.count)
            else:
                self.generate_negative_data(job.brand, job.count)
        elif job.action == JobAction.DELETE:
            self.delete_excess_data(job.brand, job.sentiment, job.count)
        self.delete_excess_data()
        self.update_status(job.brand, Status.BALANCED)
        self.report_metrics()

    def calculate_job_count(self, brand, sentiment, action):
        if action == 'generate':
            return self.samples_per_brand // 2 - self.brand_counts[brand][sentiment]
        elif action == 'delete':
            return self.brand_counts[brand][sentiment] - self.samples_per_brand // 2

    def generate_balanced_dataset(self):
        self.report_metrics()  
        self.calculate_jobs()  
        self.process_jobs() 

    def update_status(self, brand, status):
        self.brand_status[brand] = status
        self.report_metrics()

    def generate_brand_data(self, brand: str, sentiment: str, count: int = None):
        while self.brand_counts[brand][sentiment] < self.samples_per_brand // 2:
            user_prompt = self.create_prompt(brand, sentiment)
            chunks = self.samples_per_brand // 50
            for _ in range(chunks):
                self.generate_and_add_data(user_prompt, brand, sentiment)
                if self.brand_counts[brand][sentiment] >= self.samples_per_brand // 2:
                    break
            remaining = self.samples_per_brand % 50  # calculate the remaining samples
            if remaining > 0 and self.brand_counts[brand][sentiment] < self.samples_per_brand // 2:
                self.generate_and_add_data(user_prompt, brand, sentiment, remaining)  # generate the remaining samples
            self.save_generated_data()
            self.delete_excess_data()  
            self.report_metrics(brand)

    def generate_and_add_data(self, user_prompt: str, brand: str, sentiment: str, max_retries: int = 3):
        retries = 0
        while retries < max_retries:
            data = self.generate_data(user_prompt)
            data = data.split("\n")
            valid_data = [item for item in data if len(item.split("|||")) >= 3]
            if len(valid_data) > 0:
                self.add_valid_data_to_dataset(valid_data, brand, sentiment)
                break
            else:
                retries += 1
        if retries == max_retries:
            print(f"Failed to generate valid data for {brand} after {max_retries} attempts.")
        self.report_metrics(brand)

    def add_valid_data_to_dataset(self, valid_data: list, brand: str, sentiment: str):
        for item in valid_data:
            if len(self.dataset) >= self.dataset_size:
                break
            if item not in self.seen_data:
                self.dataset.append(item)
                self.seen_data.add(item)
                self.brand_counts[brand][sentiment] += 1

    def create_prompt(self, brand: str, sentiment: str) -> str:
        if brand == "nobrand":
            return self.create_nobrand_prompt()
        else:
            return self.create_brand_prompt(brand, sentiment)

    def create_nobrand_prompt(self) -> str:
        return f"""
            Create some data in the format: 
            "tweet"|||brand
            where:
                tweet is a Twitter post, 
                brand is 'nobrand' when no brand is present.
            EXAMPLES:
                "Finally finished my first marathon in years and damn, my legs feel like jelly now!"|||nobrand
                "Today has been such a mad rush, barely got a moment's peace. Is this what Mondays are like?"|||nobrand
                "Summer really hit us hard today. It's barely spring and I'm already sweating buckets!"|||nobrand
                "Spent my afternoon curled up with a really good book and a cup of tea. Can't think of a better way to unwind."|||nobrand
            Create {self.samples_per_brand} data points.
            """

    def create_positive_prompt(self, brand: str) -> str:
        if brand == "nobrand":
            return self.create_nobrand_prompt()
        else:
            return f"""
                Create some data in the format: 
                "tweet"|||brand|||sentiment 
                where:
                    tweet is a Twitter post, 
                    brand is {brand}, and
                    sentiment is 1 indicating positive sentiment of the tweet.
                    EXAMPLES:
                        "My Taco Bell was great guys"|||Taco Bell|||1
                        "Big thanks to our friends at Hattiesburg Coca-Cola for the banner celebrating R6's run to the national championship. We appreciate your support! Banner is on display in the Cook Union, stop by and see it!"|||Coca Cola|||1
                Create {self.samples_per_brand} data points.
                """

    def create_negative_prompt(self, brand: str) -> str:
        if brand == "nobrand":
            return self.create_nobrand_prompt()
        else:
            return f"""
                Create some data in the format: 
                "tweet"|||brand|||sentiment 
                where:
                    tweet is a Twitter post, 
                    brand is {brand}, and
                    sentiment is 0 indicating negative sentiment of the tweet.
                    EXAMPLES:
                        "It's taken me years, up to this moment, to realize my Gibson guitar sucks ass"|||Gibson|||0
                        "ppl at netflix are making a lot of really fucking dumb decisions like...way too often. like they got a bad idea assembly line just cranking out mistakes"|||Netflix|||0
                Create {self.samples_per_brand} data points.
                """

    def generate_positive_data(self, brand: str, count: int = None):
        while self.brand_counts[brand]["positive"] < self.samples_per_brand // 2:
            user_prompt = self.create_positive_prompt(brand)
            chunks = self.samples_per_brand // 50
            for _ in range(chunks):
                self.generate_and_add_data(user_prompt, brand, "positive")
                if self.brand_counts[brand]["positive"] >= self.samples_per_brand // 2:
                    break
            remaining = self.samples_per_brand % 50  # calculate the remaining samples
            if remaining > 0 and self.brand_counts[brand]["positive"] < self.samples_per_brand // 2:
                self.generate_and_add_data(user_prompt, brand, "positive", remaining)  # generate the remaining samples
            self.save_generated_data()
            self.delete_excess_data()  
            self.report_metrics(brand)

    def generate_negative_data(self, brand: str, count: int = None):
        while self.brand_counts[brand]["negative"] < self.samples_per_brand // 2:
            user_prompt = self.create_negative_prompt(brand)
            chunks = self.samples_per_brand // 50
            for _ in range(chunks):
                self.generate_and_add_data(user_prompt, brand, "negative")
                if self.brand_counts[brand]["negative"] >= self.samples_per_brand // 2:
                    break
            remaining = self.samples_per_brand % 50  # calculate the remaining samples
            if remaining > 0 and self.brand_counts[brand]["negative"] < self.samples_per_brand // 2:
                self.generate_and_add_data(user_prompt, brand, "negative", remaining)  # generate the remaining samples
            self.save_generated_data()
            self.delete_excess_data()  
            self.report_metrics(brand)
            
    def generate_data(self, user_prompt: str) -> str:
        wait_time = 5
        retries = 0
        while retries < 3:
            try:
                chat_completion = openai.ChatCompletion.create(
                    model=self.model,
                    temperature=0.4,
                    messages=[
                        {"role": "user", "content": user_prompt},
                    ],
                )
                return chat_completion.choices[0].message.content
            except openai.OpenAIError as e:
                if "Rate limit exceeded" in str(e) or "Bad Gateway" in str(e):
                    logging.error(f"Error: {str(e)}. Retrying in {wait_time} seconds...")
                    print(f"Error: {str(e)}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    wait_time *= 2
                    retries += 1
                else:
                    logging.error(f"Unexpected error: {str(e)}")
                    raise e
        logging.error("Failed to generate data after 3 retries.")
        raise Exception("Failed to generate data after 3 retries.")

    def load_or_initialize_dataset(self):
        if os.path.exists(self.cache_file):
            self.load_generated_data(self.cache_file)
        else:
            self.initialize_dataset()
            self.save_generated_data() 

    def initialize_dataset(self):
        self.dataset = []
        self.seen_data = set()
        self.brand_counts = {
            brand: {"positive": 0, "negative": 0} for brand in self.brands
        }
        self.brand_counts["nobrand"] = {"positive": 0, "negative": 0}

    def save_generated_data(self):
        with open(self.cache_file, "wb") as f:
            pickle.dump(self.dataset, f)

    def load_generated_data(self, filename: str):
        with open(filename, "rb") as f:
            loaded_data = pickle.load(f)
        if isinstance(loaded_data, list):
            self.dataset = loaded_data
            self.update_counts()
        else:
            print(
                "Loaded data is not in the expected format (list), initializing a new dataset."
            )
            self.initialize_dataset()

    def update_counts(self):
        for data in self.dataset:
            parts = data.split("|||")
            if len(parts) < 3:
                continue
            brand = parts[1].lower() 
            sentiment = "positive" if parts[2] == "1" else "negative"
            self.brand_counts[brand][sentiment] += 1
            
    def report_metrics(self, last_updated_brand: str = None):
        clear_output(wait=True)

        brands = []
        positive_counts = []
        negative_counts = []
        status = []

        for brand in self.brand_counts.keys():
            if brand != "nobrand":
                brands.append(brand)
                positive_counts.append(f"{self.brand_counts[brand]['positive']}/{self.samples_per_brand//2}")
                negative_counts.append(f"{self.brand_counts[brand]['negative']}/{self.samples_per_brand//2}")
                status.append(self.get_status(brand)) 

        df_brands = pd.DataFrame(
            {
                "Brands": brands,
                "Positive Counts": positive_counts,
                "Negative Counts": negative_counts,
                "Status": status,
            }
        )

        df_nobrand = pd.DataFrame(
            {
                "Brand": ["nobrand"],
                "Count": [
                    f"{self.brand_counts['nobrand']['positive'] + self.brand_counts['nobrand']['negative']}/{self.samples_per_brand}"
                ],
                "Status": [
                    Status.BALANCED.name
                    if self.brand_counts["nobrand"]["positive"]
                    + self.brand_counts["nobrand"]["negative"]
                    == self.samples_per_brand
                    else Status.QUEUED.name
                ],
            }
        )

        df_jobs = pd.DataFrame([(job.brand, job.sentiment, job.action.name , job.count) for job in self.job_queue.queue], columns=['Brand', 'Sentiment', 'Action', 'Count'])
        
        df1_styler = df_brands.style.set_table_attributes("style='display:inline; vertical-align:top'").set_caption('Brands')
        df2_styler = df_nobrand.style.set_table_attributes("style='display:inline; vertical-align:top'").set_caption('No Brand')
        df3_styler = df_jobs.style.set_table_attributes("style='display:inline; vertical-align:top'").set_caption('Jobs')

        display_html(df1_styler._repr_html_() + df2_styler._repr_html_() + df3_styler._repr_html_(), raw=True)

        
    def get_status(self, brand: str) -> Status:
        if brand is None:
            return Status.NOT_STARTED.name
        elif brand == "nobrand":
            if (
                self.brand_counts[brand]["positive"]
                + self.brand_counts[brand]["negative"]
                == self.samples_per_brand
            ):
                return Status.BALANCED.name
            else:
                return Status.QUEUED.name
        else:
            if (
                self.brand_counts[brand]["positive"]
                + self.brand_counts[brand]["negative"]
                == self.samples_per_brand
            ):
                return Status.BALANCED.name
            elif ( self.brand_counts[brand]["positive"] > 0
                or self.brand_counts[brand]["negative"] > 0
            ):
                return Status.BALANCING.name
            else:
                return Status.QUEUED.name

    def delete_excess_data(self):
        for brand in self.brands + ["nobrand"]:
            for sentiment in ["positive", "negative"]:
                while self.brand_counts[brand][sentiment] > self.samples_per_brand // 2:
                    for i in range(len(self.dataset) - 1, -1, -1):
                        parts = self.dataset[i].split("|||")
                        if len(parts) < 3:
                            continue
                        data_brand = parts[1]
                        data_sentiment = "positive" if parts[2] == "1" else "negative"
                        if data_brand == brand and data_sentiment == sentiment:
                            del self.dataset[i]
                            self.brand_counts[brand][sentiment] -= 1
                            break

In [64]:
data_generator = DataGenerator(BRANDS, samples_per_brand=100)
data_generator.calculate_jobs()
data_generator.process_jobs()

### Processing and Integrating Generated Data

The raw content generated by the GPT-4 model is processed by splitting it into three parts: 'tweet', 'brand-presence', and 'brand'. This is achieved by separating each line of the raw content on the delimiter '|||'. The processed content is then stored in a structured format as a list of dictionaries.

The processed content is converted into a pandas DataFrame and added to the existing `brands_df` DataFrame. This integration step ensures the newly generated data is combined with any current data for further analysis. Any missing values in the 'brand' column of the DataFrame are filled with 'nobrand'. This step ensures data consistency by providing a placeholder value for tweets where no brand is mentioned.

In [ ]:
# Load the data
data_generator.load_generated_data("cache/generated_data.pkl")

# Convert the data into a DataFrame
data = [item.split("|||") for item in data_generator.dataset]
brands_df = pd.DataFrame(data, columns=["tweet", "brand", "sentiment"])

NameError: name 'data_generator' is not defined

In [ ]:
print(f"brands_df.shape: {brands_df.shape}")

In [ ]:
print(f"brands_df.brand.value_counts(): {brands_df.brand.value_counts()}")

In [ ]:
print(f"brands_df.tail(6): \n{brands_df.tail(6)}")

### Brand Mention Distribution Visualization

We create a visual representation of the distribution of brand mentions within a collection of tweets. We use a bar plot to illustrate the count of tweets mentioning a brand versus those not.

In [ ]:
brand_presence = brands_df["brand"].apply(
    lambda x: "No Brand" if x == "nobrand" else "Brand"
)
print(brand_presence.value_counts())

counts = brand_presence.value_counts().reset_index()

plt.figure(figsize=(8, 8))
ax = sns.barplot(x="index", y=brand_presence.name, data=counts)
plt.title("Presence of Brand in Tweets")
plt.ylabel("Count")

for p in ax.patches:
    # ADD COUNTS TO BARS
    ax.text(
        p.get_x() + p.get_width() / 2.0,
        p.get_height(),
        "%d" % int(p.get_height()),
        fontsize=12,
        color="black",
        ha="center",
        va="bottom",
    )

# SAVE VISUALIZATION
plt.tight_layout()
plt.savefig("report/brand_count.png", dpi=300)

plt.show()

In [ ]:
brand_counts = brands_df["brand"].value_counts()
print(brand_counts)

plt.figure(figsize=(15, 12))
plt.pie(
    brand_counts,
    labels=brand_counts.index,
    autopct="%1.1f%%",
    startangle=140,
    pctdistance=0.85,
    labeldistance=1,
)
plt.axis("equal")
plt.title("Proportion of Tweets Referencing Each Brand")

# SAVE VISUALIZATION
plt.tight_layout()
plt.savefig("report/proportion_brands_in_tweets.png", dpi=300)

plt.show()

### Natural Language Toolkit (`nltk`) for Text Processing

We initialize the Natural Language Toolkit (`nltk`), a Python library for processing human language data. It offers tools for various tasks, including classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

Key parts from `nltk` are downloaded: a tokenizer, a part-of-speech tagger, a named entity chunker, a corpus of words, a multilingual lexical database, and a corpus of stopwords. 

The tokenizer helps break down the text into sentences or words, the part-of-speech tagger assigns parts of speech to individual words, and the named entity chunker identifies named entities within the text.

The corpus of words and the multilingual lexical database can be used for tasks such as spellchecking or language identification.

The stopwords corpus lists common words, known as "stop words", in several languages, including English. These words are often filtered out during text processing and rarely have significant meaning.

We generate a set of English stop words to expedite membership checking. This set will be used later in the text processing pipeline to filter out stop words from the text data.


In [ ]:
# PUNKT TOKENIZER FOR SENTENCE TOKENIZATION
nltk.download("punkt", quiet=True)

# TAGGER FOR PART-OF-SPEECH TAGGING
nltk.download("averaged_perceptron_tagger", quiet=True)

# CHUNKER FOR NAMED ENTITY RECOGNITION
nltk.download("maxent_ne_chunker", quiet=True)

# CORPUS OF WORDS FOR SPELLCHECKING OR LANGUAGE IDENTIFICATION
nltk.download("words", quiet=True)

# MULTILINGUAL LEXICAL DATABASE
nltk.download("omw-1.4", quiet=True)

# DOWNLOAD STOPWORDS CORPUS
nltk.download("stopwords", quiet=True)

# CREATE SET OF ENGLISH STOPWORDS FOR FASTER MEMBERSHIP CHECKING
STOP_WORDS = set(stopwords.words("english"))

### Preprocessing Tweets for Text Analysis

We implement a preprocessing function to clean and standardize the text in tweets. This function removes URLs and @mentions, tokenizes the text into individual words, removes stopwords and punctuation, and applies stemming to reduce words to their root form. This preprocessing function is then applied to each tweet in the dataset. A sample of the original and preprocessed tweets is displayed to visually check the preprocessing results. This step ensures the preprocessing function works as expected and the tweets are ready for further text analysis.


In [ ]:
# SET OPTIONS TO REMOVE URLS AND @MENTIONS
preprocessor.set_options(preprocessor.OPT.URL, preprocessor.OPT.MENTION)

# INITIALIZE STEMMER
STEMMER = SnowballStemmer("english")

# CONVERT STOPWORDS AND PUNCTUATION TO SETS
STOP_WORDS_SET = set(STOP_WORDS)
PUNCTUATION_SET = set(string.punctuation)


def preprocess(tweet: str) -> list:
    """
    Function to preprocess a tweet by removing URLs, @mentions, stopwords and punctuation.
    It also tokenizes and stems the tweet.

    Args:
        tweet (str): The tweet to be preprocessed.

    Returns:
        list: The preprocessed tweet as a list of stemmed tokens.
    """
    # CLEAN TWEET BY REMOVING URLS AND @MENTIONS
    cleaned = preprocessor.clean(tweet)
    # TOKENIZE TWEET INTO INDIVIDUAL WORDS
    tokens = re.findall(r"\b\w+\b", cleaned)
    # REMOVE STOPWORDS AND PUNCTUATION FROM TOKENS
    filtered = [
        word
        for word in tokens
        if word not in STOP_WORDS_SET and word not in PUNCTUATION_SET
    ]
    # APPLY STEMMING TO FILTERED TOKENS
    stemmed = [STEMMER.stem(word) for word in filtered]
    return stemmed


@measure_time("preprocessing training data")
def apply_preprocessing(df):
    """
    Function to apply preprocessing to each tweet in the DataFrame using multithreading.

    Args:
        df (DataFrame): The DataFrame containing the tweets.

    Returns:
        DataFrame: The DataFrame with the preprocessed tweets.
    """
    with ThreadPoolExecutor() as executor:
        df["processed_text"] = list(executor.map(preprocess, df.tweet))
    return df


@measure_time("saving preprocessed data")
def save_preprocessed(df, filename):
    """
    Function to save preprocessed DataFrame to a file.

    Args:
        df (DataFrame): The DataFrame to be saved.
        filename (str): The name of the file.
    """
    with open(filename, "wb") as f:
        pickle.dump(df, f)


@measure_time("loading preprocessed data")
def load_preprocessed(filename):
    """
    Function to load preprocessed DataFrame from a file.

    Args:
        filename (str): The name of the file.

    Returns:
        DataFrame: The loaded DataFrame.
    """
    with open(filename, "rb") as f:
        return pickle.load(f)


# LOAD FROM FILE IF EXISTS (DELETE FILE TO RE-PREPROCESS)
preprocessed_file = "cache/preprocessed_train.pkl"
if os.path.exists(preprocessed_file):
    tweets_df = load_preprocessed(preprocessed_file)
else:
    # USE MULTITHREADING TO APPLY PREPROCESSING
    tweets_df = apply_preprocessing(tweets_df)
    save_preprocessed(tweets_df, preprocessed_file)

# SHOW SAMPLE PROCESSED TWEETS
tweets_sample = tweets_df[tweets_df.sentiment == 1].head()
for index, row in tweets_sample.iterrows():
    print(f"ORIGINAL: {row.tweet}")
    print(f"PROCESSED: {row.processed_text}\n")

In [ ]:
# SHOW FIRST FEW ROWS OF PREPROCESSED TRAINING DATA
print("\n".join([str(text) for text in tweets_df.processed_text.head()]))

In [ ]:
@measure_time("getting word sentiment frequencies")
def get_word_sentiment_frequencies(df):
    """
    Get word-sentiment frequencies from a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the tweets and sentiment labels.

    Returns:
        dict: A dictionary with word-sentiment pairs as keys and their frequencies as values.
    """
    freqs = {}
    for y, tweet in zip(df.sentiment.values.tolist(), df.processed_text):
        for word in tweet:
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    freqs_sorted = dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))

    return freqs_sorted


freqs_sorted = get_word_sentiment_frequencies(tweets_df)

In [ ]:
print(f"NUMBER OF UNIQUE WORD-SENTIMENT PAIRS: {len(freqs_sorted)}")

In [ ]:
@measure_time("printing sample preprocessed text")
def print_around_average(freqs_sorted: Dict[Tuple[Any, int], int]) -> None:
    """
    Print the first 5 items, 5 items around the average frequency, and last 5 items in a sorted dictionary.

    Args:
        freqs_sorted (dict): The sorted dictionary.
    """
    # PRINT FIRST 5 ITEMS IN THE SORTED DICTIONARY
    print("\n".join(f"{k}: {v}" for k, v in itertools.islice(freqs_sorted.items(), 5)))

    # PRINT 5 ITEMS AROUND THE AVERAGE FREQUENCY
    # Calculate the average frequency
    avg_freq = sum(freqs_sorted.values()) / len(freqs_sorted)

    # Find the index of the first item with a frequency just below the average
    below_avg_index = next(
        i for i, v in enumerate(freqs_sorted.values()) if v < avg_freq
    )

    # Get 5 items around the average frequency
    start_index = below_avg_index - 2  # 2 items before the average
    end_index = below_avg_index + 3  # 2 items after the average

    # Convert dictionary items to a list and slice the 5 items around the average
    around_avg_items = list(freqs_sorted.items())[start_index:end_index]
    print("\n".join(f"{k}: {v}" for k, v in around_avg_items))

    # PRINT LAST 5 ITEMS IN THE SORTED DICTIONARY
    print("\n".join(f"{k}: {v}" for k, v in list(freqs_sorted.items())[-5:]))


print_around_average(freqs_sorted)

### Visualization of Sentiment Distribution for Frequently Occurring Word Stems

We create a scatter plot to analyze our dataset's 50 most often occurring word stems. Each point on the plot corresponds to a word stem, positioned according to its occurrence in positive and negative sentiments.

Handling large frequency counts can lead to complexity in graphical representation. To tackle this, we apply a logarithmic transformation to the data. This technique simplifies the visualization of large numbers, making the information easier to understand. Each point on the graph is labeled with the word stem it represents. Additionally, a line is drawn on the plot to represent an equal frequency of positive and negative sentiments. This element aids in quickly determining whether a word stem is primarily associated with positive or negative sentiments in the data.


In [ ]:
@measure_time("generating sentiment counts plot")
def generate_sentiment_counts_plot(freqs_sorted, n=50):
    # GET UNIQUE LIST OF WORDS FROM KEYS IN FREQS_SORTED
    word_list = list(set(word for word, sentiment in freqs_sorted.keys()))

    # BUILD LIST OF WORDS WITH POSITIVE AND NEGATIVE SENTIMENT COUNTS
    data = [
        [word, freqs_sorted.get((word, 1), 0), freqs_sorted.get((word, 0), 0)]
        for word in word_list
        # Remove i since it is an outlier
        if word != "i"
    ]

    # SORT DATA BY THE SUM OF POSITIVE AND NEGATIVE COUNTS AND LIMIT TO TOP 'n' MOST FREQUENT WORDS
    data = sorted(data, key=lambda x: x[1] + x[2], reverse=True)[:n]

    # PREPARE X AND Y FOR PLOT, APPLY LOG TRANSFORMATION FOR LARGE COUNTS
    x = np.log([x[1] + 1 for x in data])
    y = np.log([x[2] + 1 for x in data])

    # CREATE SCATTER PLOT FOR LOG-TRANSFORMED COUNTS
    plt.figure(figsize=(8, 8))
    sns.scatterplot(x=x, y=y)
    plt.title(f"Positive and Negative Counts of {n} Most Frequent Word Stems")
    plt.xlabel("Log Positive count")
    plt.ylabel("Log Negative count")

    # ADD ANNOTATIONS TO EACH POINT IN SCATTER PLOT, ADJUST TO AVOID OVERLAP
    texts = []
    for i in range(len(data)):
        texts.append(plt.text(x[i], y[i], data[i][0], fontsize=12))

    adjust_text(texts)

    # ADD RED LINE FROM MINIMUM TO MAXIMUM VALUE ON BOTH AXES
    min_val = min(np.min(x), np.min(y))
    max_val = max(np.max(x), np.max(y))
    plt.plot([min_val, max_val], [min_val, max_val], color="red")

    # SAVE VISUALIZATION
    plt.tight_layout()
    plt.savefig("report/positive_and_negative_counts.png", dpi=300)

    plt.show()


# Call the function
generate_sentiment_counts_plot(freqs_sorted)

## Build & Evaluate Models

### Weights & Biases (`wandb`) Setup

We use Weights & Biases (wandb) for experiment tracking. This MLOps tool allows us to compare metrics from different runs and visualize model performance.

In [ ]:
wandb.finish()
wandb.login(key=os.getenv("WANDB_API_KEY"))

### Brand Classifer

The `BrandClassifier` class is a comprehensive tool created to identify brand mentions in tweets. It operates with a specified machine-learning model and a vectorizer responsible for transforming text data into a numerical format that the model can process. Once configured, the `BrandClassifier` can train the model with given data, predict based on new data inputs, and evaluate the accuracy of the model's performance. The class structures data into training, testing, and validation sets.

In [ ]:
class BrandClassifier:
    """A generic brand classifier."""

    @measure_time("initializing classifier")
    def __init__(
        self, classifier, vectorizer: CountVectorizer = CountVectorizer()
    ) -> None:
        """
        Initializes the classifier with a vectorizer and classifier.

        Args:
            classifier: The classifier to use.
            vectorizer: The vectorizer to use. Defaults to CountVectorizer().
        """
        self.model = Pipeline([("vectorizer", vectorizer), ("classifier", classifier)])
        datetime_stamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        classifier_name = self.model.named_steps["classifier"].__class__.__name__
        wandb.init(
            project="brand-sentiment-analysis",
            name=f"{datetime_stamp}-{classifier_name}",
        )

    @measure_time("training classifier")
    def fit(self, X_train: List[str], y_train: List[str]) -> None:
        """
        Fits the model to the training data.

        Args:
            X_train: The training data.
            y_train: The training labels.
        """
        self.model.fit(X_train, y_train)

    @measure_time("predicting sentiment")
    def predict(self, X: List[str]) -> List[str]:
        """
        Predicts the labels for the given data.

        Args:
            X: The data to predict labels for.

        Returns:
            List[str]: The predicted labels.
        """
        return self.model.predict(X)

    @measure_time("evaluating classifier")
    def evaluate(self, X: List[str], y: List[str]) -> Tuple[float, float, float, float]:
        """
        Evaluates the model on the given data and returns the accuracy, precision, recall, and F1 score.

        Args:
            X: The data to evaluate the model on.
            y: The true labels for the data.

        Returns:
            Tuple[float, float, float, float]: The accuracy, precision, recall, and F1 score.
        """
        y_pred = self.predict(X)
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        f1 = f1_score(y, y_pred)
        return accuracy, precision, recall, f1

    @measure_time("printing classifier results")
    def print_scores(
        self, scores: Tuple[float, float, float, float], data_name: str
    ) -> None:
        """
        Prints the accuracy, precision, recall, and F1 score.

        Args:
            scores: The scores to print.
            data_name: The name of the data the scores are for.
        """
        accuracy, precision, recall, f1 = scores
        print(f"{data_name} Accuracy: {accuracy:.4f}")
        print(f"{data_name} Precision: {precision:.4f}")
        print(f"{data_name} Recall: {recall:.4f}")
        print(f"{data_name} F1: {f1:.4f}")

    @measure_time("finishing run")
    def finish(self, scores: Tuple[float, float, float, float]) -> None:
        """
        Finishes the wandb run and saves the trained model as an artifact.

        Args:
            scores: The scores of the model.
        """
        # INITIALIZE NEW ARTIFACT
        classifier_name = self.model.named_steps["classifier"].__class__.__name__
        accuracy, precision, recall, f1 = scores

        artifact = wandb.Artifact(
            classifier_name,
            type="model",
            description="Trained model for brand sentiment analysis",
            metadata={
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1": f1,
            },
        )

        # ADD MODEL TO ARTIFACT
        joblib.dump(self.model, "model.pkl")
        artifact.add_file("model.pkl", name="classifier_name")

        # LOG ARTIFACT TO wandb
        wandb.log_artifact(artifact)

        # DELETE TEMPORARY ARTIFACT FROM FILESYSTEM
        os.remove("model.pkl")

        # FINISH RUN
        wandb.finish()

    @measure_time("splitting data")
    def split_data(
        self, X: List[str], y: List[str]
    ) -> Tuple[List[str], List[str], List[str], List[str], List[str], List[str]]:
        """
        Splits the data into training, test, validation sets.

        Args:
            X: The data to split.
            y: The labels to split.

        Returns:
            Tuple[List[str], List[str], List[str], List[str], List[str], List[str]]: The train, test, validate data and labels.
        """
        # SPLIT DATA INTO TRAINING SET AND TEMP SET WITH 80/20 SPLIT
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # SPLIT TEMP SET INTO VALIDATION SET AND TEST SET WITH 50/50 SPLIT
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42
        )

        return X_train, y_train, X_test, y_test, X_val, y_val

    @measure_time("running classifier")
    def run(self, X: List[str], y: List[str]) -> None:
        """
        Runs the entire process from training to testing.

        Args:
            X: The data to use.
            y: The labels to use.
        """
        # SPLIT DATA
        X_train, y_train, X_test, y_test, X_val, y_val = self.split_data(X, y)

        # FIT MODEL
        self.fit(X_train, y_train)

        # EVALUATE MODEL ON VALIDATION SET
        val_scores = self.evaluate(X_val, y_val)
        self.print_scores(val_scores, "Validation")

        # EVALUATE MODEL ON TEST SET
        test_scores = self.evaluate(X_test, y_test)
        self.print_scores(test_scores, "Test")

        # FINISH RUN
        self.finish(test_scores)

### Multinomial Naive Bayes (`MultinomialNB`)

The Multinomial Naive Bayes classifier is a widely recognized and used tool in text classification. This classifier is based on Bayes' theorem, a principle in probability theory and statistics that describes the probability of an event based on prior knowledge of conditions that might be related to the event. The Multinomial Naive Bayes classifier assumes that all features, such as words in a tweet, are independent of each other. This is often called the 'naive' assumption, hence the name Naive Bayes.

This classifier is effective when dealing with features that are discrete counts. In text classification, this could be the number of times a specific word appears in a document or a tweet. The Multinomial Naive Bayes classifier requires these counts to be integers, aligning perfectly with word counts in text data. This makes the Multinomial Naive Bayes classifier an ideal choice for our text classification needs, as it can effectively handle and make predictions based on the vectorized text data.

In [ ]:
BrandClassifier(MultinomialNB()).run(brands_df.tweet, brands_df.brand_presence)

In [ ]:
# Brand classification using Multinomial Naive Bayes using brand (name) as the target
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(brands_df.tweet)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(brands_df.brand)
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
model = MultinomialNB()
model.fit(xtrain, ytrain)

predictions = model.predict(xtest)
predicted_brand_names = label_encoder.inverse_transform(predictions)

accuracy = accuracy_score(ytest, predictions)
precision_micro = precision_score(ytest, predictions, average="micro")
precision_macro = precision_score(ytest, predictions, average="macro")
precision_weighted = precision_score(ytest, predictions, average="weighted")
# recall = recall_score(ytest, predictions)
# f1 = f1_score(ytest, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (micro average): {precision_micro:.4f}")
print(f"Precision (macro average): {precision_macro:.4f}")
print(f"Precision (weighted average): {precision_weighted:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1: {f1:.4f}")

class_labels = label_encoder.inverse_transform(range(len(label_encoder.classes_)))
print(class_labels)
cm = confusion_matrix(ytest, predictions)
plt.figure(figsize=(14, 12))
sns.heatmap(
    cm, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Quick test
new_tweets = [
    "apple is the best company",
    "I love apples",
    "Apples are the worst",
    "I like apple?",
]

new_tweets_vectorized = vectorizer.transform(new_tweets)

# Predict the class using the trained model
test_prediction = model.predict(new_tweets_vectorized)

# If needed, inverse transform the predicted label to get the brand name
new_predicted_brand_name = label_encoder.inverse_transform(test_prediction)

print(new_predicted_brand_name)

### Linear Support Vector Classifier (`LinearSVC`)

The Linear Support Vector Classifier is a Support Vector Machine (SVM) often used for text classification tasks. SVMs are supervised learning methods for classification, regression, and outliers detection. The LinearSVC works by identifying the best boundary or 'hyperplane' that separates different data classes. This is achieved by maximizing the margin between the classes in the training data.

One of the strengths of LinearSVC is its ability to handle high-dimensional data. In text classification, once the text data is vectorized, it can result in a high-dimensional feature space, where each dimension corresponds to a unique word in the text.

In [ ]:
BrandClassifier(LinearSVC()).run(brands_df.tweet, brands_df.brand_presence)

### Logistic Regression

Logistic Regression is a statistical model used to predict the probability of a specific class or event. Despite its name, Logistic Regression is used for classification tasks, not regression tasks. In text classification, it's used to predict the likelihood of a specific word belonging to a particular category or class, such as whether a tweet mentions a brand.

Logistic Regression is known for its simplicity and efficiency. It uses a logistic function to model a binary dependent variable, making it a suitable choice for tasks where the output can be one of two possible outcomes. In our case, this could be whether a tweet mentions a brand or not.

In [ ]:
BrandClassifier(LogisticRegression()).run(brands_df.tweet, brands_df.brand_presence)

In [ ]:
# PRINT METRICS
metrics.print_metrics()

## Sentiment Classifier ##

## Putting It All Together ##